# Lesson 1: Getting started with RAG


Welcome to Lesson 1.

To access the `data` file, go to `File` and click on `Open`.

I hope you enjoy this course!

In [ ]:
!tar -czf Lesson1.tar.gz ./

### Import required dependencies from npm and load the API key

In [ ]:
import * as mod from "https://deno.land/std@0.213.0/dotenv/mod.ts";
const keys = await mod.load({export:true}) // read API key from .env


In [ ]:
import { 
    Document, 
    VectorStoreIndex, 
    SimpleDirectoryReader 
} from "npm:llamaindex@0.1.8"

### Load our data from a local directory

In [ ]:
const documents = await new SimpleDirectoryReader()
    .loadData({directoryPath: "./data"})

### Initialize an index

In [ ]:
const index = await VectorStoreIndex.fromDocuments(documents)

### Create a query engine 

This convenience function combines several components:
- Retriever
- Postprocessing
- Synthesizer

In [ ]:
const queryEngine = index.asQueryEngine()

### Let's ask a question!

In [ ]:
const response = await queryEngine.query({
    query: "What did the author do in college?"
})

In [ ]:
console.log(response.toString())

### But what just happened? let's break it down!

You need an:
- LLM to answer questions
- embedding model to encode them

In [ ]:
import * as llamaIndex from "npm:llamaindex@0.1.8"

In [ ]:
let customLLM = new llamaIndex.OpenAI()
let customEmbedding = new llamaIndex.OpenAIEmbedding()


 Let's put the LLM and the embedding model into a `ServiceContext` object:

In [ ]:
let customServiceContext = new llamaIndex.serviceContextFromDefaults({
    llm: customLLM,
    embedModel: customEmbedding
})

### Let's make our own prompt

In [ ]:
let customQaPrompt = function({context = "", query = ""}) {
    return `Context information is below.
        ---------------------
        ${context}
        ---------------------
        Given the context information, answer the query.
        Include a random fact about whales in your answer.\
        The whale fact can come from your training data.
        Query: ${query}
        Answer:`
}

You need a `ResponseBuilder` that uses our prompt and our service context.

In [ ]:
let customResponseBuilder = new llamaIndex.SimpleResponseBuilder(
    customServiceContext,
    customQaPrompt
)

The `responseBuilder` goes to a `synthesizer`, which also needs a service context.

In [ ]:
let customSynthesizer = new llamaIndex.ResponseSynthesizer({
    responseBuilder: customResponseBuilder,
    serviceContext: customServiceContext
})

You also need a `retriever`.

In [ ]:
let customRetriever = new llamaIndex.VectorIndexRetriever({
    index
})

The `synthesizer` and the `retriever` go to our query engine:

In [ ]:
let customQueryEngine = new llamaIndex.RetrieverQueryEngine(
    customRetriever,
    customSynthesizer
)

### Let's check the response!

In [ ]:
let response2 = await customQueryEngine.query({
    query: "What does the author think of college?"
})

In [ ]:
console.log(response2.toString())